In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(physical_devices) > 0:
   tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
from tensorflow.keras.applications import DenseNet169

In [ ]:
densenet_model = DenseNet169(weights='imagenet', include_top=False, input_shape=(256, 256, 3))

In [ ]:
densenet_model.summary()

In [ ]:
for layer in densenet_model.layers:
    layer.trainable = False

In [ ]:

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)
test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('./flowers/train',
                                                 target_size = (256, 256),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('./flowers/test',
                                            target_size = (256, 256),
                                            batch_size = 32,
                                            class_mode = 'categorical')



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Activation, Dropout, Input
from tensorflow.keras import layers


In [ ]:
model = Sequential()

model.add(densenet_model)  # 8x8
                           ## conv_base is the inception network.We are keeping it here.
model.add(layers.Dropout(0.1))

model.add(Conv2D(16, (3, 3), activation='relu'))   # 6x6
# model.add(layers.BatchNormalization())

model.add(Conv2D(16, (3, 3), activation='relu'))   # 6x6
model.add(Conv2D(5, (4, 4)))            # using 4x4 kernel to see the complete image

# Here we are Flateening our dat i.e making it one dimensional which we will feed to the network.
model.add(Flatten())
#Using softmax activation function at the last layer which is used for multi class classification
model.add(Activation('softmax'))


In [ ]:
model.summary()

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])

In [ ]:
history = model.fit(training_set,
                        #  steps_per_epoch = 500,
                         epochs = 5,
                         validation_data = test_set)    
                        #  validation_steps = 2000)

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
test_image = image.load_img('tulip.jpg', target_size = (256, 256))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = model.predict(test_image)

In [ ]:
result

In [ ]:
training_set.class_indices

In [ ]:
labels_reversed = {str(value):key for key, value in training_set.class_indices.items()}

In [ ]:
labels_reversed[str(np.argmax(result))]